# Final Project: Data processing

## Initial Data Intake

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns

import os

# Read in data
DATA_PATH = "/content/drive/MyDrive/DATASCI 207/Final Project/NBA data - raw"

In [25]:
# Function to loop through directories to read in files
def load_data(path_to_data):
    '''Load datasets
    Parameters:
    path_to_data (str): This is the path to data

    Returns:
    nba_data: A pandas dataframe
    '''
    # List all folders in data directory
    dir_list = os.listdir(path_to_data)

    # Initialize data
    nba_dfs = []
    vegas_dfs = []

    # Loop through NBA files
    for dir in dir_list:
      df = pd.read_csv(os.path.join(path_to_data, dir, "raw_scores.txt"),
                       delimiter=',', skiprows=[0], header=None)
      nba_dfs.append(df)

    # Loop through Vegas files
    for dir in dir_list:
      df = pd.read_csv(os.path.join(path_to_data, dir, "vegas.txt"),
                       delimiter=',')
      vegas_dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    nba_df = pd.concat(nba_dfs, ignore_index=True)
    vegas_df = pd.concat(vegas_dfs, ignore_index=True)

    # Name columns
    nba_df.columns = ["game_date", "game_sequence","game_id","team_id","team_abbr",
                      "team_city","game_result","pts_q1","pts_q2","pts_q3",
                      "pts_q4","pts_ot1","pts_ot2","pts_ot3","pts_ot4","pts_ot5",
                      "pts_ot6","pts_ot7","pts_ot8","pts_ot9","pts_ot10","pts_tot",
                      "fg_pct","ft_pct","fg3_pct","ast","reb","tov"]

    return nba_df, vegas_df

In [26]:
# Load data
nba_df, vegas_df = load_data(DATA_PATH)

## Pre-processing

In [27]:
# Some OT columns have no info
nba_df[['pts_ot5', 'pts_ot6', 'pts_ot7', 'pts_ot8', 'pts_ot9', 'pts_ot10']].describe()

# Drop OT columns
nba_df = nba_df.drop(['pts_ot5', 'pts_ot6', 'pts_ot7', 'pts_ot8', 'pts_ot9', 'pts_ot10'], axis=1)

In [28]:
# Show NBA columns
nba_df.columns

Index(['game_date', 'game_sequence', 'game_id', 'team_id', 'team_abbr',
       'team_city', 'game_result', 'pts_q1', 'pts_q2', 'pts_q3', 'pts_q4',
       'pts_ot1', 'pts_ot2', 'pts_ot3', 'pts_ot4', 'pts_tot', 'fg_pct',
       'ft_pct', 'fg3_pct', 'ast', 'reb', 'tov'],
      dtype='object')

In [29]:
# Show Vegas columns
vegas_df.columns

Index(['Date', 'Location', 'Team', 'OppTeam', 'TeamId', 'GameId',
       'PercentBet_ML', 'Open_Line_ML', 'Pinnacle_ML', '5dimes_ML',
       'Heritage_ML', 'Bovada_ML', 'Betonline_ML', 'Average_Line_ML',
       'Best_Line_ML', 'Worst_Line_ML', 'PercentBet_Spread',
       'Open_Line_Spread', 'Open_Odds_Spread', 'Pinnacle_Line_Spread',
       'Pinnacle_Odds_Spread', '5dimes_Line_Spread', '5dimes_Odds_Spread',
       'Heritage_Line_Spread', 'Heritage_Odds_Spread', 'Bovada_Line_Spread',
       'Bovada_Odds_Spread', 'Betonline_Line_Spread', 'Betonline_Odds_Spread',
       'Average_Line_Spread', 'Average_Odds_Spread', 'Best_Line_Spread',
       'Worst_Line_Spread', 'Best_Odds_Spread', 'Worst_Odds_Spread',
       'PercentBet_OU', 'Open_Line_OU', 'Open_Odds_OU', 'Pinnacle_Line_OU',
       'Pinnacle_Odds_OU', '5dimes_Line_OU', '5dimes_Odds_OU',
       'Heritage_Line_OU', 'Heritage_Odds_OU', 'Bovada_Line_OU',
       'Bovada_Odds_OU', 'Betonline_Line_OU', 'Betonline_Odds_OU',
       'Average_Line

In [30]:
# Select desired Vegas columns
vegas_df = vegas_df[['Location', 'TeamId', 'GameId', 'Average_Line_Spread', 'Spread', 'Result']]
vegas_df.columns = ['location', 'team_id', 'game_id', 'vegas_spread', 'spread', 'result']

# Merge data
combined_df = pd.merge(nba_df, vegas_df, on=['team_id', 'game_id'], how='left')

In [31]:
# Dimensions
print("Combined shape:", combined_df.shape)
print("NBA data shape:", nba_df.shape)
print("Vegas data shape:", vegas_df.shape)

Combined shape: (17226, 26)
NBA data shape: (17226, 22)
Vegas data shape: (17208, 6)


In [32]:
# Inspect games without spread
combined_df[combined_df['vegas_spread'].isna()]

,game_date,game_sequence,game_id,team_id,team_abbr,team_city,game_result,pts_q1,pts_q2,pts_q3,pts_q4,pts_ot1,pts_ot2,pts_ot3,pts_ot4,pts_tot,fg_pct,ft_pct,fg3_pct,ast,reb,tov,location,vegas_spread,spread,result
1698,2017-02-19,1,31600001,1610616833,EST,East NBA All Stars,0-1,53,39,47,43,0,0,0,0,182,0.569,1.000,0.373,43,50,19,NaN,NaN,NaN,NaN
1699,2017-02-19,1,31600001,1610616834,WST,West NBA All Stars,1-0,48,49,47,48,0,0,0,0,192,0.587,0.750,0.333,60,61,17,NaN,NaN,NaN,NaN
4888,2013-04-16,2,21201214,1610612754,IND,Indiana,49-32,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0,0,0,NaN,NaN,NaN,NaN
4889,2013-04-16,2,21201214,1610612738,BOS,Boston,41-40,0,0,0,0,0,0,0,0,0,0.000,0.000,0.000,0,0,0,NaN,NaN,NaN,NaN
4974,2014-11-01,1,21400027,1610612742,DAL,Dallas,2-1,32,34,20,23,0,0,0,0,109,0.517,0.579,0.240,25,44,12,away,NaN,5.0,W
4975,2014-11-01,1,21400027,1610612740,NOP,New Orleans,1-1,26,26,37,15,0,0,0,0,104,0.427,0.750,0.368,24,47,13,home,NaN,-5.0,L
6748,2015-03-06,5,21400920,1610612748,MIA,Miami,27-34,18,21,34,24,0,0,0,0,97,0.468,0.684,0.370,18,37,14,NaN,NaN,NaN,NaN
6749,2015-03-06,5,21400920,1610612764,WAS,Washington,35-27,40,27,21,11,0,0,0,0,99,0.463,0.889,0.304,24,43,15,NaN,NaN,NaN,NaN
7540,2015-11-06,6,21500080,1610612748,MIA,Miami,3-3,26,23,19,19,0,0,0,0,87,0.444,0.692,0.300,21,40,11,away,NaN,-3.0,L
7541,2015-11-06,6,21500080,1610612754,IND,Indiana,3-3,16,33,22,19,0,0,0,0,90,0.450,0.542,0.333,21,43,12,home,NaN,3.0,W


In [33]:
# Games without spread are limited (6 All-Star games + 16 add'l). Drop games.
combined_df = combined_df.dropna()

In [34]:
# Drop unnecessary columns
combined_df = combined_df.drop(['game_sequence', 'game_result', 'team_city'], axis=1)
combined_df.head()

,game_date,game_id,team_id,team_abbr,pts_q1,pts_q2,pts_q3,pts_q4,pts_ot1,pts_ot2,pts_ot3,pts_ot4,pts_tot,fg_pct,ft_pct,fg3_pct,ast,reb,tov,location,vegas_spread,spread,result
0,2016-10-25,21600001,1610612752,NYK,18,27,19,24,0,0,0,0,88,0.368,0.750,0.333,17,42,18,away,9.33,-29.0,L
1,2016-10-25,21600001,1610612739,CLE,28,20,34,35,0,0,0,0,117,0.479,0.737,0.371,31,51,14,home,-9.33,29.0,W
2,2016-10-25,21600002,1610612762,UTA,26,20,37,21,0,0,0,0,104,0.488,1.000,0.333,19,31,11,away,6.00,-9.0,L
3,2016-10-25,21600002,1610612757,POR,26,28,23,36,0,0,0,0,113,0.520,1.000,0.684,22,34,12,home,-6.00,9.0,W
4,2016-10-25,21600003,1610612759,SAS,31,33,33,32,0,0,0,0,129,0.480,0.885,0.500,25,55,13,away,8.25,29.0,W


In [37]:
# Encode result column
combined_df['result'] = combined_df['result'].apply(lambda x: 1 if x == 'W' else 0)

In [44]:
# Function for mean of statistics in last n games
def mean_last_n(series, n):
  series_shift = series.shift(1)
  result = series_shift.rolling(window=n, min_periods=0).mean()
  return result

# Function for winrate in last n games
def wr_last_n(series, n):
  series_shift = series.shift(1)
  wins = series_shift.rolling(window=n, min_periods=0).sum()
  games = series_shift.rolling(window=n, min_periods=0).count()
  wr = wins / games
  return wr

# Function to calculate team performance in last n games
def gen_last_n(df, n):
  vars = ['pts_tot', 'fg_pct', 'ft_pct', 'fg3_pct', 'ast', 'reb', 'tov']
  df = df.sort_values(by=['team_id', 'game_date'])
  for var in vars:
    df[var + '_last' + str(n)] = df.groupby('team_id')[var].apply(lambda x: mean_last_n(x, n)).reset_index(level=0, drop=True)
  df['wr_last' + str(n)] = df.groupby('team_id')['result'].apply(lambda x: wr_last_n(x, n)).reset_index(level=0, drop=True)
  return df

In [45]:
# Calculate team performance in last 5/10/20 games
combined_df = gen_last_n(combined_df, 5)
combined_df = gen_last_n(combined_df, 10)
combined_df = gen_last_n(combined_df, 20)
combined_df.head()

## EDA

In [ ]:
# Distribution of total points scored
plt.figure(figsize=(10, 6))
sns.histplot(combined_df['pts_tot'], kde=True)
plt.title('Distribution of Total Points Scored')
plt.xlabel('Total Points')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Boxplot of points scored by each quarter
plt.figure(figsize=(12, 6))
sns.boxplot(data=combined_df[['pts_q1', 'pts_q2', 'pts_q3', 'pts_q4']])
plt.title('Boxplot of Points Scored by Quarter')
plt.xlabel('Quarter')
plt.ylabel('Points')
plt.show()

In [ ]:
# Team-wise points comparison
plt.figure(figsize=(14, 8))
sns.boxplot(x='team_abbr', y='pts_tot', data=combined_df)
plt.title('Team-wise Total Points Comparison')
plt.xlabel('Team')
plt.ylabel('Total Points')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Top scoring teams
top_teams = combined_df.groupby('team_abbr')['pts_tot'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(14, 6))
sns.barplot(x=top_teams.index, y=top_teams.values)
plt.title('Top 10 Scoring Teams')
plt.xlabel('Team')
plt.ylabel('Total Points')
plt.show()

In [ ]:
# Boxplot of points for winning and losing teams
plt.figure(figsize=(12, 6))
sns.boxplot(x='win_loss', y='pts_tot', data=combined_df)
plt.title('Points Distribution for Winning and Losing Teams')
plt.xlabel('Win/Loss')
plt.ylabel('Total Points')
plt.show()

In [ ]:
# Scatter plot of assists vs. points
plt.figure(figsize=(10, 6))
sns.scatterplot(x='ast', y='pts_tot', data=combined_df)
plt.title('Assists vs. Total Points')
plt.xlabel('Assists')
plt.ylabel('Total Points')
plt.show()

In [ ]:
# Scatter plot of rebounds vs. points
plt.figure(figsize=(10, 6))
sns.scatterplot(x='reb', y='pts_tot', data=combined_df)
plt.title('Rebounds vs. Total Points')
plt.xlabel('Rebounds')
plt.ylabel('Total Points')
plt.show()

In [ ]:
# Correlation heatmap
# plt.figure(figsize=(16, 10))
# numeric_cols = combined_df.select_dtypes(include=['float64', 'int64']).columns
# sns.heatmap(combined_df[numeric_cols].corr(), annot=True, fmt='.2f', cmap='coolwarm')
# plt.title('Correlation Heatmap')
# plt.show()